In [15]:
import pandas as pd
from tqdm import tqdm
from nbs import nbs_bct
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as img
from matplotlib import cm
import matplotlib
import numpy as np
from collections import Counter
from paired_connectivity_analyzer import EEGPairedPermutationAnalyser, DrawEEG
from plotting_utils import plot_compute_sign_differences, draw_edges_by_dict, plot_reproducibility_pattern, plot_reproducibility_by_frequency

from statistics import mean

%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df = pd.read_csv('eeg_dataframe_nansfilled.csv', index_col=0)
analyzer = EEGPairedPermutationAnalyser(data_df=df, num_perm=1000, thres=0.001)

In [14]:
df

,Fp1/Fp2_1_fo,Fp1/F7_1_fo,Fp1/F3_1_fo,Fp1/Fz_1_fo,Fp1/F4_1_fo,Fp1/F8_1_fo,Fp1/T3_1_fo,Fp1/C3_1_fo,Fp1/Cz_1_fo,Fp1/C4_1_fo,...,Pz/P4_7_fz,Pz/T6_7_fz,Pz/O1_7_fz,Pz/O2_7_fz,P4/T6_7_fz,P4/O1_7_fz,P4/O2_7_fz,T6/O1_7_fz,T6/O2_7_fz,O1/O2_7_fz
abs_,-0.067816,-0.190843,0.197049,0.082941,-0.431078,-0.880805,-1.597362,-0.486399,-0.505033,-0.634386,...,-0.369614,-1.591706,-0.671730,-0.722746,-1.030530,-1.353433,-0.547397,-2.283078,-0.838246,-1.366732
agv_,-0.063064,0.149510,-1.470177,-2.307671,-4.443682,-1.542283,-1.490230,-1.053519,-1.771533,-3.276315,...,-0.399129,-1.244996,-0.463251,-0.535564,-1.440858,-1.157720,-0.681816,-1.764711,-0.533204,-1.412399
ami_,-0.456335,-0.437952,-0.133411,-0.656705,-0.498032,-1.281238,-1.952807,-1.150458,-1.865200,-1.718330,...,-0.100816,-0.595306,-0.283766,-0.117159,0.246866,-0.786120,0.194194,-0.788818,-0.051139,-0.387762
ast_,-1.161369,-1.080370,-1.011179,-1.569420,-3.917162,-2.127779,-2.692695,-2.732259,-2.059262,-3.841575,...,-0.077291,-1.674175,-0.636853,-0.970301,-0.401405,-0.866471,0.059785,-1.264972,0.262019,-0.799658
avk_,-1.344665,-3.841575,-4.795873,-3.105343,-3.917162,-2.839805,-1.620330,-3.074839,-2.087983,-1.828024,...,-0.516737,-2.283078,-0.907005,-0.545027,-1.792309,-2.550227,-1.661937,-0.818822,-0.491047,-0.869325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vls_,0.309057,-0.588057,-0.190843,-0.417361,-0.231765,0.252898,-0.403681,-0.639324,-0.600152,-0.349243,...,-0.707283,-1.194784,-0.712422,-0.671730,-0.535564,-0.764710,-0.295050,-1.090605,-0.725338,-0.684346
voo_,-0.437952,-1.143244,-0.353766,-0.424215,-0.588057,-1.460305,-1.785331,-1.260946,-1.209979,-1.421794,...,-0.451732,-1.056841,-0.447133,-0.631922,-0.186278,-0.970301,-0.144968,-1.465228,-0.258922,-0.694503
zab_,-1.281238,-2.911332,-2.936540,-2.654844,-3.136956,-4.602049,-1.183529,-2.600973,-3.772040,-1.828024,...,-1.209979,-1.603051,-0.371880,-0.281508,0.047082,-1.183529,-0.479438,-1.547648,-0.788818,-0.112499
zak_,-0.074926,-0.421929,-0.265701,-0.491047,-2.936540,-0.829889,-4.091480,-2.911332,-1.037052,-1.536948,...,-0.177134,-1.872822,-0.408238,-0.474806,-1.221522,-1.252940,-0.456335,-2.795185,-1.417085,-1.076978


In [18]:
open_name = []
for j in range(1,8,1):
    open_name += [f'{analyzer.channel_bivar[i]}_{j}_fo' for i in range(len(analyzer.channel_bivar))]

df_open = analyzer.df[open_name]

In [19]:
closed_name = []
for j in range(1,8,1):
    closed_name += [f'{analyzer.channel_bivar[i]}_{j}_fz' for i in range(len(analyzer.channel_bivar))]

df_closed = analyzer.df[closed_name]

### $x, y$ должны быть типа NxNxP np.ndarray
x : matrix representing the first population with P subjects, must be symmetric.

y : matrix representing the second population with Q subjects, must be symmetric.

наш датасет нужно развернуть, сконкатинировать по людям по закрытым и открытым глазам

In [20]:
df_open

,C3/C4_1_fo,C3/Cz_1_fo,C3/O1_1_fo,C3/O2_1_fo,C3/P3_1_fo,C3/P4_1_fo,C3/Pz_1_fo,C3/T4_1_fo,C3/T5_1_fo,C3/T6_1_fo,...,T4/T5_7_fo,T4/T6_7_fo,T5/O1_7_fo,T5/O2_7_fo,T5/P3_7_fo,T5/P4_7_fo,T5/Pz_7_fo,T5/T6_7_fo,T6/O1_7_fo,T6/O2_7_fo
abs_,-0.034329,0.553366,-0.595306,-0.746215,0.026977,-0.124134,-0.117159,-1.237112,-1.053519,-1.027283,...,-2.600973,-1.961210,-0.872186,-1.986904,-1.969694,-2.442131,-1.888262,-3.016859,-2.399592,-0.810578
agv_,-1.165031,-0.276993,-1.017594,-1.649858,-0.481757,-1.515901,-0.927753,-2.773702,-0.712422,-3.999957,...,-2.886832,-2.692695,-0.454033,-2.712253,-0.378682,-1.371205,-1.563930,-1.183529,-2.068733,-0.684346
ami_,-0.869325,-0.100816,-1.731344,-2.502260,-0.474806,-1.475151,-0.717576,-2.078306,-3.591649,-2.654844,...,-2.427721,-0.612314,-0.973397,-1.580497,-1.515901,-2.583729,-1.813569,-1.510711,-1.637934,-0.982732
ast_,-1.455406,-1.237112,-1.040327,-1.510711,-0.671730,-3.016859,-1.125417,-5.397938,-0.843845,-1.936232,...,-1.626161,-1.828024,-1.403095,-1.637934,-1.542283,-3.397766,-3.169760,-1.850146,-1.521119,0.119778
avk_,-0.247615,-0.114830,-1.465228,-1.515901,-1.366732,-0.921790,-1.445683,-2.013357,-1.393877,-1.380211,...,-2.213160,-2.752735,-1.104399,-1.978258,-2.962499,-3.489314,-2.936540,-1.668036,-1.820768,-1.426525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vls_,0.352012,0.505687,0.411902,0.054691,0.806311,0.265080,0.211445,-0.022234,0.562421,-0.058302,...,-2.962499,-2.636516,-0.733135,-1.318814,-0.545027,-1.510711,-1.213813,-2.004452,-1.187268,-0.748844
voo_,-0.263441,0.252898,-0.756757,-0.895296,-0.547397,-0.597727,-0.486399,-1.260946,-0.863622,-1.465228,...,-3.169760,-1.136079,-1.191019,-1.952807,-1.172392,-2.031437,-1.699147,-2.863002,-1.526366,-0.371880
zab_,-0.629461,-0.412798,-0.007616,-0.945811,0.062339,-0.786120,0.024482,-1.850146,-0.578427,-1.187268,...,-2.583729,-0.743591,-0.936749,-1.314569,-0.717576,-1.744546,-0.973397,-1.194784,-0.493374,-0.256661
zak_,-0.733135,-0.666708,-0.609876,-1.217660,-0.022234,-0.641797,-0.512050,-4.443682,-0.709851,-2.235857,...,-3.538977,-2.712253,-0.915856,-2.158842,-0.791520,-2.040615,-1.455406,-2.886832,-1.952807,-1.001621


In [21]:
#abs_, open eyes, delta(1) band
dct_open_delta = {}  
for i in range(len(df_open.index)):
    dct_open_delta['%sopen_delta' % df_open.index[i]] = df_open.iloc[i,0:171]  

In [22]:
abs_open_delta = dct_open_delta[list(dct_open_delta.keys())[0]]
agv_open_delta = dct_open_delta[list(dct_open_delta.keys())[1]]

In [23]:
electrodes = set()
for i in range(len(abs_open_delta.index)):
    electrodes.add(abs_open_delta.index[i].split("/")[0])
    electrodes.add(abs_open_delta.index[i].split("/")[1].split("_")[0])

In [24]:
electrd = list(electrodes)
electrd.sort()

In [25]:
electrd

['C3',
 'C4',
 'Cz',
 'F3',
 'F4',
 'F7',
 'F8',
 'Fp1',
 'Fp2',
 'Fz',
 'O1',
 'O2',
 'P3',
 'P4',
 'Pz',
 'T3',
 'T4',
 'T5',
 'T6']

In [26]:
df_abs_open_delta = pd.DataFrame(index=pd.Index(electrd))
df_abs_open_delta = df_abs_open_delta.reindex(columns = electrd)    

In [27]:
df_agv_open_delta = pd.DataFrame(index=pd.Index(electrd))
df_agv_open_delta = df_agv_open_delta.reindex(columns = electrd)    

In [28]:
abs_open_delta_values = list()
for i in range(171):
    abs_open_delta_values.append(f"{abs_open_delta.index[i]} is {abs_open_delta[i]}")
abs_open_delta_valuesNEW = []
for i in range(len(abs_open_delta_values)):
    abs_open_delta_valuesNEW.append(abs_open_delta_values[i].split())

In [29]:
agv_open_delta_values = list()
for i in range(171):
    agv_open_delta_values.append(f"{agv_open_delta.index[i]} is {agv_open_delta[i]}")
agv_open_delta_valuesNEW = []
for i in range(len(agv_open_delta_values)):
    agv_open_delta_valuesNEW.append(agv_open_delta_values[i].split())

In [30]:
for k in range(len(abs_open_delta_valuesNEW)):
    for i in range(19):
        for j in range(19):
            if all(i in abs_open_delta_valuesNEW[k][0] for i in list([df_abs_open_delta.index[i], df_abs_open_delta.columns[j]])):
                df_abs_open_delta.loc[df_abs_open_delta.index[i], df_abs_open_delta.columns[j]]=float(abs_open_delta_valuesNEW[k][2])
df_abs_open_delta.values[[np.arange(df_abs_open_delta.shape[0])]*2] = 1

In [31]:
for k in range(len(agv_open_delta_valuesNEW)):
    for i in range(19):
        for j in range(19):
            if all(i in agv_open_delta_valuesNEW[k][0] for i in list([df_agv_open_delta.index[i], df_agv_open_delta.columns[j]])):
                df_agv_open_delta.loc[df_agv_open_delta.index[i], df_agv_open_delta.columns[j]]=float(agv_open_delta_valuesNEW[k][2])
df_agv_open_delta.values[[np.arange(df_agv_open_delta.shape[0])]*2] = 1

In [32]:
P = np.ndarray((2,19,19))

In [33]:
df_closed
#abs_, open eyes, delta(1) band
dct_close_delta = {}  
for i in range(len(df_closed.index)):
    dct_close_delta['%sclose_delta' % df_closed.index[i]] = df_closed.iloc[i,0:171]  
abs_close_delta = dct_close_delta[list(dct_close_delta.keys())[0]]
agv_close_delta = dct_close_delta[list(dct_close_delta.keys())[1]]
df_abs_close_delta = pd.DataFrame(index=pd.Index(electrd))
df_abs_close_delta = df_abs_close_delta.reindex(columns = electrd)  
df_agv_close_delta = pd.DataFrame(index=pd.Index(electrd))
df_agv_close_delta = df_agv_close_delta.reindex(columns = electrd)
abs_close_delta_values = list()
for i in range(171):
    abs_close_delta_values.append(f"{abs_close_delta.index[i]} is {abs_close_delta[i]}")
abs_close_delta_valuesNEW = []
for i in range(len(abs_close_delta_values)):
    abs_close_delta_valuesNEW.append(abs_close_delta_values[i].split())
agv_close_delta_values = list()
for i in range(171):
    agv_close_delta_values.append(f"{agv_close_delta.index[i]} is {agv_close_delta[i]}")
agv_close_delta_valuesNEW = []
for i in range(len(agv_close_delta_values)):
    agv_close_delta_valuesNEW.append(agv_close_delta_values[i].split())
for k in range(len(abs_close_delta_valuesNEW)):
    for i in range(19):
        for j in range(19):
            if all(i in abs_close_delta_valuesNEW[k][0] for i in list([df_abs_close_delta.index[i], df_abs_close_delta.columns[j]])):
                df_abs_close_delta.loc[df_abs_close_delta.index[i], df_abs_close_delta.columns[j]]=float(abs_close_delta_valuesNEW[k][2])
df_abs_close_delta.values[[np.arange(df_abs_close_delta.shape[0])]*2] = 1
for k in range(len(agv_close_delta_valuesNEW)):
    for i in range(19):
        for j in range(19):
            if all(i in agv_close_delta_valuesNEW[k][0] for i in list([df_agv_close_delta.index[i], df_agv_close_delta.columns[j]])):
                df_agv_close_delta.loc[df_agv_close_delta.index[i], df_agv_close_delta.columns[j]]=float(agv_close_delta_valuesNEW[k][2])
df_agv_close_delta.values[[np.arange(df_agv_close_delta.shape[0])]*2] = 1

In [34]:
Q = np.ndarray((2,19,19))
Q[0]=df_abs_close_delta
Q[1]=df_agv_close_delta
Q.shape

(2, 19, 19)

In [37]:
%%time
#nbs только для abs и agv для открытых глаз delta ритм
nbs_bct(x=P.reshape(19,19,2), y=Q.reshape(19,19,2), thresh=0.05)

max component size is 140
estimating null distribution with 1000 permutations
permutation 0 of 1000.  p-value so far is 1.000
permutation 100 of 1000.  p-value so far is 0.386
permutation 200 of 1000.  p-value so far is 0.393
permutation 300 of 1000.  p-value so far is 0.372
permutation 400 of 1000.  p-value so far is 0.354
permutation 500 of 1000.  p-value so far is 0.343
permutation 600 of 1000.  p-value so far is 0.349
permutation 700 of 1000.  p-value so far is 0.354
permutation 800 of 1000.  p-value so far is 0.338
permutation 900 of 1000.  p-value so far is 0.334
permutation 999 of 1000.  p-value so far is 0.335
CPU times: user 19.3 s, sys: 372 ms, total: 19.7 s
Wall time: 22.2 s


(array([0.335]),
 array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.],
        [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 0.],
        [1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         1., 1., 1.],
        [1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
         0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
         1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
         1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 